In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, TimeDistributed
from tensorflow.keras.utils import to_categorical

In [3]:
#--- Hằng số dữ liệu ---
OUTPUT_CSV = './Outputs/training-feature.csv'

#--- Hằng số mô hình chuỗi thời gian ---
SEQUENCE_LENGTH = 30    # Số lượng frame liên tiếp được coi là 1 mẫu huấn luyện (ví dụ: 1 giây @ 30 FPS)
NUM_FEATURES = 9        # Số đặc trưng đầu vào (EAR, MAR, PITCH, YAW, ..., UNRELIABLE_POSE)
NUM_CLASSES = 4         # Số trạng thái đầu ra (0: awake, 1: sleep, 2: yawning, 3: passed_out)

In [ ]:
#--- Tải và Chuẩn hóa Dữ liệu ---

# Tải file .csv
df = pd.read_csv(OUTPUT_CSV)

# 1. Tách Đặc trưng (X) và Nhãn (y)
X = df[['EAR', 'MAR', 'PITCH', 'YAW', 'ROLL', 'D_SLUMP', 'R_TILT', 'EYE_CL', 'UNRELIABLE_POSE']].values
y = df[['Label']].values

# 2. Chuẩn hoá (Scaling) đặc trưng
# Đảm bảo các đặc trưng nằm trong khoảng từ 0 đến 1 (hoặc -1 đến 1)
scaler = MinMaxScaler(feature_range=(-1,1))
X_scaled = scaler.fit_transform(X)

# 3. Mã hoá Nhãn (One-hot Encoding)
# Chuyển nhãn số (0, 1, 2, 3) thành vector (ví dụ: 2 -> [0, 0, 1, 0])

y_encoded = to_categorical(y, num_classes=NUM_CLASSES)

print(f"Total frames has been scaled: {X_scaled.shape[0]}")

In [ ]:
#--- Tạo Chuỗi Thời gian (Sliding Window) ---
"""
Đây là bước quan trọng nhất. Chúng ta cần chuyển mảng 2D (Frame x Features) 
thành mảng 3D (Sample x Time Steps x Features) để LSTM có thể xử lý.
"""
def create_sequence(X, y, seq_length):
    X_seq, y_seq = [] , []

    # Lặp qua tất cả các frame, trừ seq_length frame cuối cùng
    for i in range(len(X) - seq_length + 1):
        # Lấy một cửa sổ (window) X dài seq_length frame
        window_X = X[i : i + seq_length]

        # Lấy nhãn y cho frame cuối cùng của cửa sổ (Nhãn cuối cho chuỗi đó)
        window_y = y_encoded[i + seq_length - 1]

        X_seq.append(window_X)
        y_seq.append(window_y)

    return np.array(X_seq), np.array(y_seq)

# Tạo dữ liệu chuỗi
X_sequences, y_sequences = create_sequences(X_scaled, y, SEQUENCE_LENGTH)

print(f"Final training data has shape: {X_sequences.shape}") # Vd: (2000, 30, 9)
print(f"Label output has shape: {y_sequences.shape}") #Vd (2000, 4)

# Tách tập Train và Test (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(
    X_sequences, y_sequences, test_size=0.2, random_state=42, shuffle=True
)
print(f"Shape X_train: {X_train.shape}")

In [ ]:
#--- Xây dựng và Huấn luyện Mô hình CNN-LSTM ---
"""
Kiến trúc này kết hợp CNN để học các mẫu ngắn hạn trong chuỗi (ví dụ: nháy mắt)
và LSTM để học sự phụ thuộc dài hạn (ví dụ: gục đầu kéo dài).
"""

def build_cnn_lstm_model(input_shape, num_classes):
    model = Sequential()

    # 1. Lớp CNN 1D (Học các mẫu cục bộ trên trục thời gian)
    model.add(ConV1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))

    # 2. Lớp LSTM (Học sự phụ thuộc chuỗi dài hạn)
    # return_sequences=False vì đây là lớp cuối cùng trước Dense
    